In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

In [2]:
from __future__ import unicode_literals, print_function, division

# import basic lib
import re
import math
import random
import string
import unicodedata
import numpy as np
from io import open

# import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

# import loss func
import masked_cross_entropy

# check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_idx = 0
EOS_idx = 1
UNK_idx = 2
PAD_idx = 3

USE_CUDA = True

class Preprocessor:
    '''
    class for preprocessing
    '''
    def __init__(self, name):
        '''
        initialize vocab and counter
        '''
        self.name = name
        self.w2idx = {"<sos>" : 0, "<eos>" : 1, "<unk>" : 2, "<pad>" : 3}
        self.counter = {}
        self.idx2w = {0: "<sos>", 1: "<eos>", 2:"<unk>", 3:"<pad>"}
        self.num = 4

    def SentenceAdder(self, sentence):
        '''
        Add a sentence to dataset
        '''
        for word in sentence.split(' '):
            self.WordAdder(word)

    def WordAdder(self, word):
        '''
        Add single word to dataset and update vocab and counter
        '''
        if word in self.w2idx:
            self.counter[word] += 1
        else:
            self.w2idx[word] = self.num
            self.counter[word] = 1
            self.idx2w[self.num] = word
            self.num += 1
            
    def trim(self, min_count=1):
        '''
        Trim to remove non-frequent word
        '''
        keep = []
        for k, v in self.counter.items():
            if v >= min_count: keep.append(k)
        print(self.name+':')
        print('Total words', len(self.w2idx))
        print('After Trimming', len(keep))
        print('Keep Ratio %', 100 * len(keep) / len(self.w2idx))
        self.w2idx = {"<sos>" : 0, "<eos>" : 1, "<unk>" : 2, "<pad>" : 3}
        self.counter = {}
        self.idx2w = {0: "<sos>", 1: "<eos>", 2:"<unk>", 3:"<pad>"}
        self.num = 4
        for w in keep:
            self.WordAdder(w)

In [4]:
def Uni2Ascii(s):
    '''
    transfer from unicode to ascii
    '''
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

def StrCleaner(s):
    '''
    trim, delete non-letter and lowercase string
    '''
    s = Uni2Ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def DataReader(path, lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(path, encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    #pairs = [[StrCleaner(s) for s in l.split('<------>')] for l in lines]
    pairs = [[s.lower() for s in l.split('<------>')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Preprocessor(lang2)
        output_lang = Preprocessor(lang1)
    else:
        input_lang = Preprocessor(lang1)
        output_lang = Preprocessor(lang2)

    return input_lang, output_lang, pairs

In [5]:
MIN_LENGTH = 3
MAX_LENGTH = 50

def filterPair(p):
    '''
    Filter to get expected pairs with specific length
    '''
    return MIN_LENGTH <= len(p[0].split(' ')) <= MAX_LENGTH and \
        MIN_LENGTH <= len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(path, lang1, lang2, reverse=True):
    input_lang, output_lang, pairs = DataReader(path, lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.SentenceAdder(pair[0])
        output_lang.SentenceAdder(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.num)
    print(output_lang.name, output_lang.num)
    return input_lang, output_lang, pairs


src, tgt, pairs = prepareData('data/train.txt', 'english', 'chinese')
src.trim()
tgt.trim()
print(random.choice(pairs))

Reading lines...
Read 90000 sentence pairs
Trimmed to 77083 sentence pairs
Counting words...
Counted words:
chinese 46064
english 30499
chinese:
Total words 46064
After Trimming 46060
Keep Ratio % 99.99131642931573
english:
Total words 30499
After Trimming 30495
Keep Ratio % 99.9868848158956
['这次 会议 将 为 今年 " 十 加 三 " 领导人 会晤 做 准备 .', 'this meeting will prepare for the meeting of leaders of the " 10 plus 3 " this year .']


In [7]:
def sentence2idx(preprocessor, sentence):
    '''
    Read sentence and translate into word index plus eos
    '''
    return [SOS_idx] + [preprocessor.w2idx[w] if w in preprocessor.w2idx \
            else UNK_idx for w in sentence.split(' ')] + [EOS_idx]

def pad(seq, max_len):
    '''
    Add padding to sentence with different length
    '''
    seq += [PAD_idx for i in range(max_len - len(seq))]
    return seq

def random_batch(src, tgt, pairs, batch_size, batch_idx):
    '''
    Randomly generate batch data
    '''
    inputs, target = [], []
    
    # Choose batch
    for s in pairs[batch_idx*batch_size:(batch_idx+1)*batch_size]:
        inputs.append(sentence2idx(src, s[0]))
        target.append(sentence2idx(tgt, s[1]))
        
    # Sort by length
    seq_pairs = sorted(zip(inputs, target), key=lambda p: len(p[0]), reverse=True)
    inputs, target = zip(*seq_pairs)
    
    # Obtain length of each sentence and pad
    input_lens = [len(s) for s in inputs]
    input_max = max(input_lens)
    input_padded = [pad(s, input_max) for s in inputs]
    target_lens = [len(s) for s in target]
    target_max = max(target_lens)
    target_padded = [pad(s, target_max) for s in target]

    # Create Variable
    if USE_CUDA:
        input_vars = Variable(torch.LongTensor(input_padded).cuda()).transpose(0, 1)
        input_lens = Variable(torch.LongTensor(input_lens).cuda())
        target_vars = Variable(torch.LongTensor(target_padded).cuda()).transpose(0, 1)
        target_lens = Variable(torch.LongTensor(target_lens).cuda())
    else:
        input_vars = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
        input_lens = Variable(torch.LongTensor(input_lens))
        target_vars = Variable(torch.LongTensor(target_padded)).transpose(0, 1)
        target_lens = Variable(torch.LongTensor(target_lens))

    return input_vars, input_lens, target_vars, target_lens

In [28]:
class Encoder(nn.Module):
    '''
    Define encoder and forward process
    '''
    def __init__(self, dim_input, dim_embed, dim_hidden, num_layers, dropout):
        super(Encoder, self).__init__()
        self.dim_input = dim_input
        self.dim_hidden = dim_hidden
        self.dim_embed = dim_embed
        self.embed = nn.Embedding(dim_input, dim_embed)
        self.cell = nn.GRU(dim_embed, dim_hidden, 
                          num_layers, dropout=dropout, 
                          bidirectional=True)
        
    def forward(self, inputs, inputs_lens, hidden=None):
        '''
        We need to sum the outputs since bi-diretional is used
        '''
        #print('e')
        embedded = self.embed(inputs)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, inputs_lens)
        outputs, hidden = self.cell(packed, hidden)
        outputs, output_lengths = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.dim_hidden] + \
                    outputs[:, :, self.dim_hidden:]
        return outputs, hidden


class Attention(nn.Module):
    '''
    Define attention mechanism
    '''
    def __init__(self, dim_hidden):
        super(Attention, self).__init__()
        self.dim_hidden = dim_hidden
        # 2*dim_hidden is needed since bi-direction is used
        self.attn = nn.Linear(2*self.dim_hidden, dim_hidden)
        self.v = nn.Parameter(torch.rand(dim_hidden))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        #print('a')
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        scores = self.score(h, encoder_outputs)
        return F.relu(scores).unsqueeze(1)

    def score(self, hidden, encoder_outputs):
        e = F.softmax(self.attn(torch.cat([hidden, encoder_outputs], 2)),dim=1)
        e = e.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        e = torch.bmm(v, e)
        return e.squeeze(1)


class Decoder(nn.Module):
    '''
    Define decoder with attention
    '''
    def __init__(self, dim_embed, dim_hidden, dim_output, num_layers, dropout):
        super(Decoder, self).__init__()
        self.dim_embed = dim_embed
        self.dim_hidden = dim_hidden
        self.dim_output = dim_output
        self.num_layers = num_layers

        self.embed = nn.Embedding(dim_output, dim_embed)
        self.dropout = nn.Dropout(dropout, inplace=True)
        self.attention = Attention(dim_hidden)
        self.cell = nn.GRU(dim_hidden + dim_embed, dim_hidden,
                          num_layers, dropout=dropout)
        self.out = nn.Linear(2*dim_hidden, dim_output)

    def forward(self, inputs, last_hidden, encoder_outputs):
        
        
        embedded = self.embed(inputs).unsqueeze(0)  # (1,B,N)
        embedded = self.dropout(embedded)
        
        print('input size ' + str(inputs.size()))
        print('input type ' + str(inputs.type()))
        print('emmb size ' + str(embedded.size()))
        print('emmb type ' + str(embedded.type()))
        
        
        # Calculate attention weights and apply to encoder outputs
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  # (B,1,N)
        context = context.transpose(0, 1)  # (1,B,N)
        # Combine embedded input word and attended context, run through RNN

        rnn_input = torch.cat([embedded, context], 2)
        output, hidden = self.cell(rnn_input, last_hidden)
        output = output.squeeze(0)  # (1,B,N) -> (B,N)
        context = context.squeeze(0)
        output = self.out(torch.cat([output, context], 1))
        output = F.log_softmax(output, dim=1)
        return output, hidden, attn_weights

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_len, tgt, tgt_len, teacher_forcing_ratio=0.5):
        batch_size = src.size(1)
        max_len = tgt.size(0)
        vocab_size = self.decoder.dim_output
        if USE_CUDA:
            outputs = Variable(torch.zeros(max_len, batch_size, vocab_size).cuda())
        else:
            outputs = Variable(torch.zeros(max_len, batch_size, vocab_size))
        encoder_output, hidden = self.encoder(src, src_len)
        hidden = hidden[:self.decoder.num_layers]
        # Put <sos> at first position
        if USE_CUDA:
            output = Variable(tgt.data[0, :].cuda())
        else:
            output = Variable(tgt.data[0, :])
        for t in range(1, max_len):
            output, hidden, attn_weights = self.decoder(
                    output, hidden, encoder_output)
            outputs[t] = output
            # Randomly choose whether to use teacher force or not
            is_teacher = random.random() < teacher_forcing_ratio
            top1 = output.data.max(1)[1]
            if USE_CUDA:
                output = Variable(tgt.data[t].cuda() if is_teacher else top1.cuda())
            else:
                output = Variable(tgt.data[t] if is_teacher else top1)
        return outputs
    
    def inference(self, src, src_len, max_len = MAX_LENGTH):
        pred_idx = []
        batch_size = src.size(1)
        vocab_size = self.decoder.dim_output
        if USE_CUDA:
            outputs = Variable(torch.zeros(max_len, batch_size, vocab_size).cuda())
        else:
            outputs = Variable(torch.zeros(max_len, batch_size, vocab_size))
        
        encoder_output, hidden = self.encoder(src, src_len)
        hidden = hidden[:self.decoder.num_layers]
        # Put <sos> at first position
        if USE_CUDA:
            output = Variable(src.data[0, :].cuda())
        else:
            output = Variable(src.data[0, :])
        pred_idx.append(SOS_idx)
        for t in range(1, max_len):
            output, hidden, attn_weights = self.decoder(
                    output, hidden, encoder_output)
            outputs[t] = output
            top1 = output.data.max(1)[1]
            pred_idx.append(top1.item())
            if USE_CUDA:
                output = Variable(top1.cuda())
            else:
                output = Variable(top1)
            if top1 == EOS_idx: break
        return outputs, pred_idx


In [30]:
batch_size = 128
hidden_size = 512
embed_size = 256
encoder_n_layers = 2
decoder_n_layers = 1 
# n_layers = 4
width = 5
encoder_test = Encoder(src.num, embed_size, hidden_size, encoder_n_layers, dropout=0.2)
decoder_test = Decoder(embed_size, hidden_size, tgt.num, decoder_n_layers, dropout=0.2)

In [31]:
net = Seq2Seq(encoder_test,decoder_test).cuda()
# opt = optim.Adam(net.parameters(),lr=0.0001)
net.load_state_dict(torch.load('./saved_model_1.pt'))


In [32]:
batch_size = 100

test_src, test_tgt, test_pairs = prepareData('data/test.txt', 'english', 'chinese')
test_src.trim()
test_tgt.trim()
print(random.choice(test_pairs))

test_src.w2idx, test_src.idx2w, test_src.num = src.w2idx, src.idx2w, src.num
test_tgt.w2idx, test_tgt.idx2w, test_tgt.num = tgt.w2idx, tgt.idx2w, tgt.num
test_pairs.sort(key=lambda x: len(x[0].split()))

input_batches, input_lengths,\
    target_batches, target_lengths = random_batch(test_src,test_tgt,test_pairs,batch_size,5)

Reading lines...
Read 10000 sentence pairs
Trimmed to 8572 sentence pairs
Counting words...
Counted words:
chinese 17294
english 13049
chinese:
Total words 17294
After Trimming 17290
Keep Ratio % 99.9768705909564
english:
Total words 13049
After Trimming 13045
Keep Ratio % 99.96934631006208
['进入 新 千年 之际 , 各国 政治家 , 思想家 和 各种 组织 都 在 思考 未来 .', 'as we enter the new millennium , politicians , thinkers , and organizations of all kinds are pondering the future .']


In [33]:
input_batches, input_lengths,\
    target_batches, target_lengths = random_batch(test_src,test_tgt,test_pairs,batch_size,15)

In [34]:
# print(input_batches.size())
# print(target_batches.size())

print(input_batches[:,1])
print(target_batches[:,1])

tensor([   0,  191,   73,   34,   22,   28,  543,   66,  260, 3190,  245,  566,
          28, 1401,   16,    1], device='cuda:0')
tensor([   0,   68,  123,  124,   10,  564,   15,   14,   31,  291,   58,   30,
           4,  203,  204,   99, 1519,  134,   24,    1,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3], device='cuda:0')


In [47]:
class BeamSearch(nn.Module):
    '''
    Implement BeamSearch for testing
    '''
    
    def __init__(self, encoder, decoder, width):
        super(BeamSearch, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.width = width
        
    def forward(self, src, src_len, width, max_len):
        pred_result = []
        vocab_size = self.decoder.dim_output        
        encoder_output, hidden = self.encoder(src, src_len)
        hidden = hidden[:self.decoder.num_layers]
        #output is in the shape of batch_size * dict size
        beam_best = Variable(torch.zeros(width, max_len)).cuda()
        prob_best = Variable(torch.zeros(width, 1)).cuda()
        #beam options, all possible width * width options
        beam_options = Variable(torch.zeros(width * width, max_len)).cuda()
        prob_options = Variable(torch.zeros(width * width, 1)).cuda()

        if USE_CUDA:
            outputs = Variable(torch.zeros(width * width, vocab_size).cuda())
            output = Variable(src.data[0, :].cuda())
            new_output = Variable(src.data[0, :].cuda())
            best_output = Variable(torch.zeros(width, vocab_size).cuda())
        else:
            outputs = Variable(torch.zeros(width * width, vocab_size))
            output = Variable(src.data[0, :])
            new_output = Variable(src.data[0, :])
            best_output = Variable(torch.zeros(width, vocab_size))
        
        output, hidden, attn_weights = self.decoder(
            output, hidden, encoder_output)
        
        #val: prob, idx: index
        #fill topk best items
        val, idx = output.data.topk(k = width, dim = 1)
#         copy.data = torch.tensor(output, requires_grad=True).to(device='cuda:0')
#         copy = output.clone()
#         print('copy size' + str(copy.size()))
#         print('copy data' + str(copy[0][idx[0][4]]))
        print(idx[0,:]) #tensor([  4, 261, 285,  40, 466], device='cuda:0')
        print(val[0,:]) #tensor([-0.0034, -6.9214, -7.3048, -7.4925, -8.7961], device='cuda:0')
    
        #generate beams
        for i in range(width):
            beam_best[i][1] = idx[0][i]
            prob_best[i] = val[0][i].exp() #since prob log,softmax from -10 to -12
            best_output[i] = output
        
        print('theshape ' + str(best_output.size()))
        for t in range(2, max_len):
            cnt = 0
            for i in range(width):
                curr_prob = prob_best[i]
#                 print('valval ' + str(best_output[i][0][idx[0][i]]))
                if i != 0:
                    best_output[i][idx[0][i]] = -50 #disable the top i-1 item to guarantee ith item work
# x_cuda = Variable(x, requires_grad=True).cuda()    
                #print('copy size ' + str(copy.size()))
                new_output, hidden, attn_weights = self.decoder(
                        best_output[i], hidden, encoder_output)
                
                new_output.data = new_output.data.exp() * curr_prob #use add because this is log here
                val1, idx1 = new_output.data.topk(k = width, dim = 1)
                for j in range(width):
                    beam_options[cnt] = beam_best[i]
                    beam_options[cnt][t] = idx1[0][j]
                    prob_options[cnt] = val1[0][j]
                    outputs[cnt] = new_output
                    cnt += 1
            
            topVal, topInx = prob_options.topk(k = width, dim = 0)
#             print(topVal.size())
#             print(topInx.size())

            for j in range(topInx.size(0)):
#                 prob_best[j] = prob_options[topInx[j][0]] 
                prob_best[j] = topVal[0][j]
                beam_best[j] = beam_options[topInx[0][j]]  
                best_output[j] = outputs[topInx[0][j]]

        best_index = prob_best.max(0).item()
        return beam_options[best_index].item()

In [48]:
net = BeamSearch(net.encoder,net.decoder, width).cuda()


In [49]:
# import bleu
for test_idx in range(60):
#     pred = net(input_batches[:,test_idx].reshape(input_lengths[0].item(),1),input_lengths[0].reshape(1), 5)
    print("input_batches " + str(input_batches.size()))
    print("input_length " + str(input_lengths))
    print("target_batches " + str(target_batches.size()))
    print("target_lengths " + str(target_lengths))
    pred = net(input_batches[:,test_idx].reshape(input_lengths[0].item(),1), input_lengths[0].reshape(1), width, input_lengths[0].reshape(1))

    inp = ' '.join([src.idx2w[t] for t in input_batches[:,test_idx].cpu().numpy()])
    mt = ' '.join([tgt.idx2w[t] for t in pred if t!= PAD_idx])
    ref = ' '.join([tgt.idx2w[t] for t in target_batches[:,test_idx].cpu().numpy()])
    print('INPUT:\n' + inp)
    print('REF:\n' + ref)
    print('PREDICTION:\n' + mt)
    print('BLEU = %f' % bleu([mt],[[ref]],4))
    print("------")


input_batches torch.Size([16, 100])
input_length tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16], device='cuda:0')
target_batches torch.Size([41, 100])
target_lengths tensor([24, 20, 18, 26, 21, 20, 21, 14, 31, 15, 23, 36, 21, 17, 21, 27, 24, 27,
        24, 16, 27, 25, 22, 20, 31, 15, 23, 18, 22, 23, 28, 19, 25, 22, 18, 19,
        41, 25, 25, 19, 26, 18, 23, 23, 23, 15, 24, 27, 20, 21, 23, 19, 15, 21,
        28,  9, 28, 22, 19, 25, 28, 25, 16, 27, 15, 25, 24, 17, 19, 27, 33, 14,
        22, 17, 19, 16, 19, 23, 24, 29, 19, 27, 23, 17, 21, 27, 23, 20, 23, 15,
        20, 13, 21, 22, 20, 20, 2

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got CUDAFloatTensor instead (while checking arguments for embedding)